<a href="https://colab.research.google.com/github/GuiVdwinden/greeneye/blob/master/BasePreprocessor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Load images and dataframe
--------------

In [ ]:
import pandas as pd
#from google.colab import files
from google.colab import drive
drive.mount('/content/drive') #let's connect to google drive

Mounted at /content/drive


In [ ]:
#@title
!unzip /content/drive/MyDrive/GreenEye/datasets/train-jpg.zip #where the data is stored

Streaming output truncated to the last 5000 lines.
  inflating: train-jpg/train_5499.jpg  
  inflating: train-jpg/train_55.jpg  
  inflating: train-jpg/train_550.jpg  
  inflating: train-jpg/train_5500.jpg  
  inflating: train-jpg/train_5501.jpg  
  inflating: train-jpg/train_5502.jpg  
  inflating: train-jpg/train_5503.jpg  
  inflating: train-jpg/train_5504.jpg  
  inflating: train-jpg/train_5505.jpg  
  inflating: train-jpg/train_5506.jpg  
  inflating: train-jpg/train_5507.jpg  
  inflating: train-jpg/train_5508.jpg  
  inflating: train-jpg/train_5509.jpg  
  inflating: train-jpg/train_551.jpg  
  inflating: train-jpg/train_5510.jpg  
  inflating: train-jpg/train_5511.jpg  
  inflating: train-jpg/train_5512.jpg  
  inflating: train-jpg/train_5513.jpg  
  inflating: train-jpg/train_5514.jpg  
  inflating: train-jpg/train_5515.jpg  
  inflating: train-jpg/train_5516.jpg  
  inflating: train-jpg/train_5517.jpg  
  inflating: train-jpg/train_5518.jpg  
  inflating: train-jpg/train_5519

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/GreenEye/datasets/BalancedData.csv') #let's load the dataframe

## Delete 'Clear' tag

In [ ]:
#now we remove the tag 'clear' so we balance it a little bit

alltags = [ 'cloudy', 'haze', 'partly cloudy',        #removed the clear!!    
           'agriculture', 'artisinal mine', 'bare ground', 'blooming', 
           'blow down', 'cultivation', 'habitation', 'primary', 'road', 
        'selective logging', 'conventional mine', 'slashu burn','water']

#remove the clear tag
data.drop('clear', axis = 1, inplace =True)

In [ ]:
data.head(1)

,image_name,tags,taglist,cloudy,haze,partly cloudy,agriculture,artisinal mine,bare ground,blooming,blow down,cultivation,habitation,primary,road,selective logging,conventional mine,slashu burn,water,result_id
0,train_0,haze primary,"['haze', 'primary']",0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,100000000100000


## tweak the names of the files so it's compatible

In [ ]:
data['image_name'] = data['image_name'] + '.jpg'

In [ ]:
data['image_name'].head(1)

0    train_0.jpg
Name: image_name, dtype: object

# Preprocess data: Tensorflow ImageDataGenerator
-----------------------

## preprocessing

[the documents for imagedatagenerator](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator#random_transform)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#instantiate the data generator and input the transformatons you want

datagen = ImageDataGenerator(rescale = 1./255)  #rescale the imagest, it's going to be needed later


test_datagen=ImageDataGenerator(rescale = 1./255)

## Image Selection and Processing 

### choosing number of pictures to train/val/test

In [ ]:
totaln = 20000  #just for training sake

In [ ]:
# let's define a train/test/val ratios and set
train_split = 0.8  #train/test ratio
share_train = 0.8   #train/ val ratio
share_val = 1- share_train  # percentage of train set going to the validation set

#let's calculate the number of images per set
n_train = int(totaln*train_split*share_train) 
n_val = int(totaln*train_split*share_val)
n_test = int(totaln *(1-train_split))

print('train size: ' + str(n_train))
print('valid size: ' + str(n_val))
print('n_test: ' + str(n_test))

train size: 12800
valid size: 3199
n_test: 3999


In [ ]:
totaln = n_train + n_val + n_test
print(f'total number taken : {totaln}')

total number taken : 19998


### Process, agument and prepare the data for the model 

In [ ]:
#if you're using VGG16 uncomment the following import
#from tensorflow.keras.applications.vgg16 import preprocess_input

In [ ]:
#TRAIN SET
train_generator = datagen.flow_from_dataframe(
                                            dataframe=data[0:n_train],  #taking the amount of images to train specified above 
                                            directory="/content/train-jpg",
                                            x_col="image_name",
                                            y_col=alltags,
                                            batch_size=32,
                                            shuffle=True,
                                            class_mode="raw",  #look into this later
                                            target_size=(128,128),
#attention, if you need vgg, uncomment this:  #preprocessing_function=preprocess_input   #(this is the needed preprocessing for VGG16)
)

Found 12800 validated image filenames.


In [ ]:
#VALIDATION SET
valid_generator=test_datagen.flow_from_dataframe(
                                                dataframe=data[n_train:n_train+n_val], #taking the amount of validation images specified above 
                                                directory="/content/train-jpg",
                                                x_col="image_name",
                                                y_col=alltags,
                                                batch_size=32,
                                                shuffle=True,
                                                class_mode="raw",
                                                target_size=(128,128))

Found 3199 validated image filenames.


In [ ]:
#TEST SET
test_generator=test_datagen.flow_from_dataframe(
                                                dataframe=data[n_train+n_val:n_train+n_val+n_test], #taking the amount of test images specified above 
                                                directory="/content/train-jpg",
                                                x_col="image_name",
                                                y_col=alltags,
                                                batch_size=32,
                                                shuffle=True,
                                                class_mode="raw",
                                                target_size=(128,128)
)

Found 3999 validated image filenames.


In [ ]:
#EXTRA, AN IMAGE FOR PREDICTIONS!
image_number = 22789   #just any image
sample_predict=test_datagen.flow_from_dataframe(
                                                dataframe=data[image_number:image_number+1],
                                                directory="/content/train-jpg",
                                                x_col="image_name",
#                                                y_col=alltags, # I don't remember why I commented this  out, but let's just leave it as is
                                                batch_size=32,
                                                seed=42,
                                                shuffle=False,
                                                class_mode=None,
                                                target_size=(128,128)
)


Found 1 validated image filenames.


In [ ]:
sample_predict[0].shape

(1, 128, 128, 3)

# MODEL

# Evaluation
---------

In [ ]:
import matplotlib.pyplot as plt
def plot_loss_accuracy(history):
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='best')
    plt.show()
    
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='best')
    plt.show()

    plt.plot(history.history['fbeta_score_K'])
    plt.plot(history.history['val_fbeta_score_K'])
    plt.title('Model f beta_score_K')
    plt.ylabel('fbeta_score_K')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='best')
    plt.show()

plot_loss_accuracy(history)

# Prediction
-----

In [ ]:
prediction= model.predict(sample_predict[0])

In [ ]:
prediction

In [ ]:
#its real properties are:
data.iloc[image_number,3:-1]

# Model Storage

In [ ]:
# from tensorflow.keras import models

# models.save_model(model, 
#                   'YOUR_MODEL_NAME AND FILETYPE',  #filepath
#                   overwrite=True, 
#                   include_optimizer=True, #important 
#                   save_format= 'h5'   
# )     

In [ ]:
# from google.colab import files
# files.download("Model-02-12")

In [ ]:
# from tensorflow.keras.preprocessing import image
# from tensorflow.keras.models import load_image
# import numpy as np


In [ ]:
#EXTRA image preprocessing from scratch so you can upload any image and test it

# model = load_model(filepath_to_model_folder)
# img = image.load_img(img_path, target_size=(128, 128))
# img_array = image.img_to_array(img)
# img_tensor = np.expand_dims(img_array, axis=0)
# preds = model.predict(x)
